# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al, psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE
    # and then back ...
    supercell = ase_atoms(lattice, atoms, positions) * (repeat, 1, 1)
    lattice   = load_lattice(supercell)
    positions = load_positions(supercell)
    atoms = fill(Al, length(positions))

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(lattice, atoms, positions; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
ase_atoms(setup.model).write("al_supercell.png")

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
is_converged = DFTK.ScfConvergenceDensity(1e-4)  # Flag convergence based on density
self_consistent_field(aluminium_setup(1); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298485410037                   -0.85    5.2
  2   -8.300204196370       -2.76       -1.25    1.1
  3   -8.300448350803       -3.61       -1.89    3.5
  4   -8.300462679589       -4.84       -2.78    1.5
  5   -8.300464540631       -5.73       -3.09    3.2
  6   -8.300464588804       -7.32       -3.25    3.6
  7   -8.300464615064       -7.58       -3.39    1.1
  8   -8.300464629803       -7.83       -3.53    5.6
  9   -8.300464639508       -8.01       -3.71    1.0
 10   -8.300464640862       -8.87       -3.77    2.1
 11   -8.300464642999       -8.67       -3.97    1.0
 12   -8.300464644014       -8.99       -4.65    1.2


In [4]:
self_consistent_field(aluminium_setup(2); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -16.67382832690                   -0.70    5.6
  2   -16.67792951689       -2.39       -1.15    1.4
  3   -16.67916137266       -2.91       -1.87    4.0
  4   -16.67924802629       -4.06       -2.65    3.4
  5   -16.67928462429       -4.44       -3.07    5.5
  6   -16.67928620895       -5.80       -3.49    1.9
  7   -16.67928621740       -8.07       -4.05    1.0


In [5]:
self_consistent_field(aluminium_setup(4); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32628681813                   -0.56    7.0
  2   -33.33418563911       -2.10       -1.00    1.2
  3   -33.33598982855       -2.74       -1.73    4.8
  4   -33.33613706292       -3.83       -2.65    7.5
  5   -33.33680272590       -3.18       -2.25    6.5
  6   -33.33683564501       -4.48       -2.31    1.1
  7   -33.33694281967       -3.97       -3.24    3.4
  8   -33.33694391482       -5.96       -3.84    4.5
  9   -33.33694382681   +   -7.06       -3.83    1.5
 10   -33.33694392603       -7.00       -4.76    4.4


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); is_converged, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298516705636                   -0.85    5.0
  2   -8.300264783498       -2.76       -1.59    1.0
  3   -8.300434360414       -3.77       -2.44    4.8
┌ Warning: Eigensolver not converged
│   iterations =
│    8-element Vector{Int64}:
│     100
│       4
│       4
│       6
│       4
│       4
│       5
│       3
└ @ DFTK ~/work/DFTK.jl/DFTK.jl/src/scf/self_consistent_field.jl:34
  4   -8.300330756356   +   -3.98       -2.19   16.2
  5   -8.300464524617       -3.87       -3.72    4.5
  6   -8.300464595650       -7.15       -3.95    3.2
  7   -8.300464634799       -7.41       -4.29    1.8


In [7]:
self_consistent_field(aluminium_setup(4); is_converged, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32508324371                   -0.56    6.4
  2   -33.32120775518   +   -2.41       -1.27    1.0
  3   -19.54048510345   +    1.14       -0.50    8.1
  4   -33.31562665900        1.14       -1.78    8.0
  5   -33.24876299700   +   -1.17       -1.35    2.8
  6   -33.25272153492       -2.40       -1.60    3.9
  7   -33.20047370184   +   -1.28       -1.49    5.1
  8   -33.33595985977       -0.87       -2.40    3.9
  9   -33.33654100193       -3.24       -2.61    3.1
 10   -33.33663360447       -4.03       -2.50    2.9
 11   -33.33692739209       -3.53       -3.31    2.2
 12   -33.33694191009       -4.84       -3.58    3.2
 13   -33.33694308682       -5.93       -3.80    2.4
 14   -33.33694381681       -6.14       -4.11    2.6


For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.